In [1]:
import mdtraj as mdt
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [65]:
t = mdt.load('1qy1.pdb')
sel = t.topology.select('resid 0 to 10')
tsel = t.atom_slice(sel)
print(tsel)

<mdtraj.Trajectory with 1 frames, 84 atoms, 11 residues, and unitcells>


In [66]:
print(tsel.topology.residue(0).name)

GLU


In [67]:
x = tsel.xyz[0].flatten()
l = len(x)
dx = np.zeros((l, l))
for i in range(l):
    for j in range(l):
        dx[i, j] = x[i] - x[j]

In [68]:
sij = squareform(pdist(x.reshape((-1, 3))))

In [69]:
def Hij(i, j, dx, sij, gamma, cut):
    
    H = np.zeros((3, 3))
    if i != j:
        fac = -gamma/sij[i, j]**2
        for k in range(3):
            ii1 = 3 * i + k
            ii2 = 3 * j + k
            for l in range(3):
                jj1 = 3 * i + l
                jj2 = 3 * j + l
                H[k, l] = fac*dx[ii1, ii2]*dx[jj1, jj2]
    else:
        js = np.where(sij[i] < cut)[0]
        for k in range(3):
            ii1 = 3 * i + k
            for l in range(3):
                jj1 = 3 * i + l
                s = 0.0
                for j in js:
                    if j != i:
                        ii2 = 3 * j + k
                        jj2 = 3 * j + l
                        s += dx[ii1, ii2]*dx[jj1, jj2] / sij[i, j]**2
                H[k, l] = s * gamma
    return H

In [70]:
H = np.zeros_like(dx)
gamma = 2.0
cut = 1.0
for i in range(l//3):
    ii = i * 3
    for j in range(l//3):
        jj = j * 3
        H[ii:ii+3, jj:jj+3] = Hij(i, j, dx, sij, gamma, cut)

In [71]:
w, v = np.linalg.eigh(H)

In [72]:
print(w)

[-45.643904   -20.20655155 -15.39515533 -12.19036361  -9.82448444
  -6.0045583    1.08157334   1.3654564    1.82005456   2.03258601
   2.28734692   2.42134169   2.83094561   3.17458844   3.37629109
   3.48342992   3.81351776   4.40071933   4.47713612   4.65042815
   5.50760725   5.65780573   5.75616868   5.84812561   6.08330987
   6.23387114   6.48695421   6.64612554   6.87875122   7.20971484
   7.3741129    7.79627927   7.84275982   8.18033415   8.24260417
   9.21454252   9.50466424  10.29693438  10.30667216  10.52581013
  10.61059808  10.99584464  11.51060053  12.03112279  12.45686264
  12.52763954  12.85521964  12.88272847  12.96044006  13.11220484
  13.3564261   13.36845738  13.4805451   14.00740091  14.3633061
  14.71440855  14.83342844  15.12041958  15.29477811  15.41738965
  15.59647262  15.6426389   16.1677005   16.46873551  16.63515521
  17.04609497  17.31215119  17.41761804  17.71020255  17.81443247
  17.99417235  18.09219936  18.56412951  18.61643369  18.81562483
  19.289841

In [73]:
print(v[:,-1])

[ 0.01977713  0.00441675 -0.04979846  0.02633269  0.00629596 -0.05622393
  0.0222345   0.01430648 -0.06585296  0.01451681  0.01223722 -0.07108354
  0.03466291 -0.00503455 -0.06201066  0.03011689 -0.01398636 -0.07281082
  0.01815292 -0.01439033 -0.05782442  0.01194694 -0.01791284 -0.05809936
  0.01849464 -0.00940197 -0.04809889  0.02852794  0.02493333 -0.0703692
  0.02343272  0.03899927 -0.07981642  0.0494021   0.06101726 -0.12968659
  0.0650853   0.0533229  -0.11736855  0.01563809  0.04187454 -0.0614298
  0.00964059  0.03071585 -0.05020188  0.0071084   0.03335457 -0.04397758
  0.00802745  0.03992047 -0.04121539  0.0050051   0.02901373 -0.04129506
  0.04397703  0.0724658  -0.15597341  0.04450675  0.07505598 -0.14185416
  0.00292687  0.08984585 -0.09428035 -0.01871176  0.09817593 -0.09978414
  0.03529457  0.03172146 -0.12637807 -0.00489141  0.09005495 -0.04317845
 -0.03588346  0.10674165 -0.01265883 -0.05339521  0.07578005  0.01439633
 -0.03493552  0.05402035  0.0250506  -0.0172643   0.1

In [74]:
tsel[0].save('tsel0.pdb')

In [75]:
sfac = 0.01
n_frames = 10
n_atoms = tsel.n_atoms
traj = np.zeros((n_frames, n_atoms, 3))
for i in range(10):
    x = tsel.xyz[0].copy()
    for j in range(8):
        k = -(j+2)
        r = np.random.rand() - 0.5
        x += v[:,k].reshape((n_atoms, 3)) * w[k] * sfac * r
    traj[i] = x

In [76]:
tnew = mdt.Trajectory(traj, tsel.topology)
tnew.save('tnew.pdb')
rmsd = mdt.rmsd(tnew, tsel[0])
print(rmsd)

[0.06329782 0.07844521 0.06733258 0.08995488 0.07274853 0.09343169
 0.07675215 0.05859298 0.0649004  0.07692649]


In [58]:
print(np.random.rand() - 0.5)

-0.3443324554355758


In [59]:
print(traj[0])

[[2.86550784 2.94976234 4.94087267]
 [2.86289644 2.97501898 4.78065443]
 [3.01627898 2.88805008 4.69550037]
 [3.08038545 2.97554231 4.76444149]
 [2.84329581 3.16334033 4.73143005]
 [2.94522333 3.25142264 4.74665928]
 [2.9617126  3.2839303  4.88520241]
 [3.03713679 3.31355834 4.90027332]
 [2.88486242 3.24217463 4.9503684 ]]


In [60]:
print(tsel.xyz[0])

[[2.8737 2.9865 4.9224]
 [2.8615 2.9813 4.7726]
 [2.9915 2.9267 4.7118]
 [3.1014 2.9588 4.7564]
 [2.8322 3.121  4.7158]
 [2.9444 3.2243 4.7399]
 [2.9597 3.269  4.8871]
 [3.0604 3.3376 4.9176]
 [2.8725 3.2376 4.9718]]


In [63]:
print(tnew.xyz[0])

[[-0.07863665 -0.16610432  0.11916113]
 [-0.08124804 -0.14084768 -0.04105711]
 [ 0.07213449 -0.22781658 -0.12621117]
 [ 0.13624096 -0.14032435 -0.05727005]
 [-0.10084867  0.04747367 -0.09028149]
 [ 0.00107884  0.13555598 -0.07505226]
 [ 0.01756811  0.16806364  0.06349087]
 [ 0.09299231  0.19769168  0.07856178]
 [-0.05928206  0.12630796  0.12865686]]
